# 🎯 Stability Selection with Google Drive Checkpointing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kstawiski/OmicSelector2/blob/main/examples/05_stability_selection.ipynb)

Learn robust feature selection with automatic checkpointing to Google Drive.

**What You'll Learn:**
- 🔄 Stability selection theory and practice
- 💾 Automatic checkpoint saving to Drive (resume if disconnected!)
- 📊 Stability paths visualization
- 🎯 Threshold selection strategies
- ⚡ Comparison with single-run methods

**Why Stability Selection?**
- More robust than single feature selection
- Better generalization to new data
- Identifies truly important features
- Reduces false discoveries

**Estimated Time**: 30-40 minutes  
**Prerequisites**: Basic feature selection knowledge

---

## 🚀 Setup

In [ ]:
# Install
!pip install -q git+https://github.com/kstawiski/OmicSelector2.git

# Mount Drive
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=False)

BASE_DIR = '/content/drive/MyDrive/OmicSelector2'
os.makedirs(f'{BASE_DIR}/checkpoints/stability_selection', exist_ok=True)
os.makedirs(f'{BASE_DIR}/results/stability', exist_ok=True)
os.makedirs(f'{BASE_DIR}/plots/stability', exist_ok=True)

print(f"✅ Drive mounted: {BASE_DIR}")
print("\n💡 Checkpoints will be saved automatically!")
print("   → If Colab disconnects, you can resume from last checkpoint")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from omicselector2.features.classical.random_forest import RandomForestSelector
from omicselector2.features.classical.lasso import LassoSelector
from omicselector2.features.stability import StabilitySelector

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Libraries loaded!")

## 📊 Load or Create Data

In [ ]:
# Try to load from Drive first
data_path = f"{BASE_DIR}/data/processed/splits/X_train.csv"
labels_path = f"{BASE_DIR}/data/processed/splits/y_train.csv"

if os.path.exists(data_path) and os.path.exists(labels_path):
    print("📂 Loading preprocessed data from Drive...")
    X = pd.read_csv(data_path, index_col=0)
    y = pd.read_csv(labels_path)['response'].values
    print(f"✅ Loaded: {X.shape}")
else:
    print("🧬 Creating sample dataset...")
    X, y = make_classification(
        n_samples=300,
        n_features=500,
        n_informative=30,
        n_redundant=20,
        random_state=42
    )
    X = pd.DataFrame(X, columns=[f"GENE_{i:04d}" for i in range(X.shape[1])])
    print(f"✅ Created: {X.shape}")

# Split for demonstration
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain: {X_train.shape[0]} samples")
print(f"Test: {X_test.shape[0]} samples")

## 🎯 Stability Selection with Checkpointing

This will run 100 bootstraps with automatic checkpointing to Drive.

In [ ]:
# Checkpoint management
checkpoint_file = f"{BASE_DIR}/checkpoints/stability_selection/checkpoint.pkl"

def save_checkpoint(data, filepath):
    """Save checkpoint to Drive."""
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)
    print(f"💾 Checkpoint saved: {filepath}")

def load_checkpoint(filepath):
    """Load checkpoint from Drive."""
    if os.path.exists(filepath):
        with open(filepath, 'rb') as f:
            return pickle.load(f)
    return None

# Check for existing checkpoint
checkpoint = load_checkpoint(checkpoint_file)

if checkpoint is not None:
    print(f"✅ Found checkpoint! Resuming from bootstrap {checkpoint['last_bootstrap']}")
    resume = input("Resume from checkpoint? (y/n): ").lower() == 'y'
    if not resume:
        checkpoint = None
        print("Starting fresh...")
else:
    print("No checkpoint found. Starting fresh...")

# Run stability selection
print("\n🔄 Running Stability Selection...")
print("   • Bootstraps: 100")
print("   • Sample fraction: 80%")
print("   • Method: Random Forest")
print("   • Checkpoints: Every 10 bootstraps\n")

selector = StabilitySelector(
    base_selector=RandomForestSelector(n_estimators=50, task='classification'),
    n_bootstraps=100,
    threshold=0.6,
    sample_fraction=0.8,
    n_features_to_select=50,
    random_state=42,
    checkpoint_dir=f"{BASE_DIR}/checkpoints/stability_selection",
    checkpoint_every=10  # Save every 10 bootstraps
)

selector.fit(X_train, y_train)

print("\n✅ Stability selection complete!")
print(f"   • Selected {len(selector.selected_features_)} stable features")

# Save results to Drive
results_path = f"{BASE_DIR}/results/stability/stability_results.pkl"
with open(results_path, 'wb') as f:
    pickle.dump(selector, f)
print(f"\n💾 Results saved to: {results_path}")

## 📊 Visualize Stability Paths

In [ ]:
# Get stability scores
stability_scores = selector.stability_scores_
selected_features = selector.selected_features_

# Create stability scores DataFrame
stability_df = pd.DataFrame({
    'feature': X_train.columns,
    'stability_score': stability_scores,
    'selected': [feat in selected_features for feat in X_train.columns]
}).sort_values('stability_score', ascending=False)

print("🎯 Top 20 Most Stable Features:")
print("=" * 60)
display(stability_df.head(20))

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Stability score distribution
axes[0, 0].hist(stability_scores, bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(selector.threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold = {selector.threshold}')
axes[0, 0].set_xlabel('Stability Score', fontsize=12)
axes[0, 0].set_ylabel('Number of Features', fontsize=12)
axes[0, 0].set_title('Stability Score Distribution', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Top 30 features
top_30 = stability_df.head(30)
colors = ['green' if sel else 'gray' for sel in top_30['selected']]
axes[0, 1].barh(range(len(top_30)), top_30['stability_score'], color=colors, alpha=0.7)
axes[0, 1].set_yticks(range(len(top_30)))
axes[0, 1].set_yticklabels(top_30['feature'])
axes[0, 1].set_xlabel('Stability Score', fontsize=12)
axes[0, 1].set_title('Top 30 Features by Stability', fontsize=14, fontweight='bold')
axes[0, 1].axvline(selector.threshold, color='red', linestyle='--', linewidth=2)
axes[0, 1].invert_yaxis()
axes[0, 1].grid(True, alpha=0.3, axis='x')

# 3. Cumulative selection
sorted_scores = np.sort(stability_scores)[::-1]
cumsum = np.cumsum(sorted_scores)
axes[1, 0].plot(cumsum, linewidth=2)
axes[1, 0].axvline(len(selected_features), color='red', linestyle='--', linewidth=2, label=f'Selected: {len(selected_features)}')
axes[1, 0].set_xlabel('Number of Features', fontsize=12)
axes[1, 0].set_ylabel('Cumulative Stability Score', fontsize=12)
axes[1, 0].set_title('Cumulative Stability Score', fontsize=14, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Selected vs not selected comparison
selected_scores = stability_df[stability_df['selected']]['stability_score']
not_selected_scores = stability_df[~stability_df['selected']]['stability_score']
axes[1, 1].boxplot([selected_scores, not_selected_scores], labels=['Selected', 'Not Selected'])
axes[1, 1].set_ylabel('Stability Score', fontsize=12)
axes[1, 1].set_title('Selected vs Not Selected Features', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plot_path = f"{BASE_DIR}/plots/stability/stability_analysis.png"
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
print(f"\n💾 Stability plots saved to: {plot_path}")
plt.show()

# Save stability scores
scores_path = f"{BASE_DIR}/results/stability/stability_scores.csv"
stability_df.to_csv(scores_path, index=False)
print(f"💾 Stability scores saved to: {scores_path}")

## 🔍 Compare with Single-Run Selection

In [ ]:
print("🔍 Comparing Stability Selection vs Single-Run\n" + "="*60 + "\n")

# Single-run Random Forest
single_rf = RandomForestSelector(
    n_estimators=100,
    n_features_to_select=50,
    task='classification',
    random_state=42
)
single_rf.fit(X_train, y_train)
single_features = single_rf.selected_features_

# Compare overlap
stable_features = set(selected_features)
single_features_set = set(single_features)

overlap = stable_features & single_features_set
only_stable = stable_features - single_features_set
only_single = single_features_set - stable_features

print(f"📊 Feature Set Comparison:")
print(f"   • Stability selection:  {len(stable_features)} features")
print(f"   • Single-run RF:        {len(single_features_set)} features")
print(f"   • Overlap:              {len(overlap)} features ({len(overlap)/len(stable_features)*100:.1f}%)")
print(f"   • Only in stability:    {len(only_stable)} features")
print(f"   • Only in single-run:   {len(only_single)} features")

# Visualize overlap
from matplotlib_venn import venn2

plt.figure(figsize=(10, 6))
venn2([stable_features, single_features_set], 
      set_labels=('Stability Selection', 'Single-Run RF'),
      set_colors=('skyblue', 'lightcoral'),
      alpha=0.7)
plt.title('Feature Selection Method Comparison', fontsize=14, fontweight='bold')

venn_path = f"{BASE_DIR}/plots/stability/methods_comparison.png"
plt.savefig(venn_path, dpi=300, bbox_inches='tight')
print(f"\n💾 Comparison plot saved to: {venn_path}")
plt.show()

# Test set evaluation
from sklearn.ensemble import RandomForestClassifier as SklearnRF
from sklearn.metrics import roc_auc_score

print("\n🎯 Test Set Performance:")
print("="*60)

# Stability features
X_train_stable = X_train[list(stable_features)]
X_test_stable = X_test[list(stable_features)]
clf_stable = SklearnRF(n_estimators=100, random_state=42)
clf_stable.fit(X_train_stable, y_train)
y_pred_stable = clf_stable.predict_proba(X_test_stable)[:, 1]
auc_stable = roc_auc_score(y_test, y_pred_stable)

# Single-run features
X_train_single = X_train[list(single_features_set)]
X_test_single = X_test[list(single_features_set)]
clf_single = SklearnRF(n_estimators=100, random_state=42)
clf_single.fit(X_train_single, y_train)
y_pred_single = clf_single.predict_proba(X_test_single)[:, 1]
auc_single = roc_auc_score(y_test, y_pred_single)

print(f"Stability Selection AUC: {auc_stable:.4f}")
print(f"Single-Run RF AUC:       {auc_single:.4f}")
print(f"\n{'✅ Stability is better!' if auc_stable > auc_single else '⚠️  Single-run is better this time'}")
print(f"Difference: {abs(auc_stable - auc_single):.4f}")

## 📝 Create Final Report

In [ ]:
from datetime import datetime

report = f"""
═══════════════════════════════════════════════════════════════
    STABILITY SELECTION REPORT - OmicSelector2
═══════════════════════════════════════════════════════════════

Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Location: {BASE_DIR}

CONFIGURATION
───────────────────────────────────────────────────────────────
Base Method:          Random Forest (50 trees)
Number of Bootstraps: {selector.n_bootstraps}
Sample Fraction:      {selector.sample_fraction*100:.0f}%
Stability Threshold:  {selector.threshold}
Random Seed:          {selector.random_state}

RESULTS
───────────────────────────────────────────────────────────────
Total Features:       {X_train.shape[1]}
Stable Features:      {len(stable_features)}
Selection Rate:       {len(stable_features)/X_train.shape[1]*100:.1f}%

Top 10 Stable Features:
{', '.join(stability_df.head(10)['feature'].tolist())}

COMPARISON WITH SINGLE-RUN
───────────────────────────────────────────────────────────────
Feature Overlap:      {len(overlap)}/{len(stable_features)} ({len(overlap)/len(stable_features)*100:.1f}%)
Test AUC (Stability): {auc_stable:.4f}
Test AUC (Single):    {auc_single:.4f}
Improvement:          {(auc_stable - auc_single):.4f}

FILES SAVED
───────────────────────────────────────────────────────────────
Results:              {BASE_DIR}/results/stability/
Plots:                {BASE_DIR}/plots/stability/
Checkpoints:          {BASE_DIR}/checkpoints/stability_selection/

KEY INSIGHTS
───────────────────────────────────────────────────────────────
✓ Stability selection completed with {selector.n_bootstraps} bootstraps
✓ Checkpoints saved every 10 bootstraps (resumable if disconnected)
✓ {len(stable_features)} features consistently selected
✓ {'Better' if auc_stable > auc_single else 'Similar'} generalization vs single-run

═══════════════════════════════════════════════════════════════
"""

print(report)

report_path = f"{BASE_DIR}/results/stability/stability_report.txt"
with open(report_path, 'w') as f:
    f.write(report)

print(f"\n💾 Report saved to: {report_path}")
print(f"\n✅ All results saved to Google Drive!")
print(f"📂 Access at: {BASE_DIR}")

## 🎓 Summary

### What You've Learned

✅ **Stability Selection**: More robust than single-run methods  
✅ **Checkpointing**: Resume interrupted analyses from Drive  
✅ **Stability Paths**: Visualize feature consistency  
✅ **Threshold Selection**: Choose appropriate stability thresholds  
✅ **Performance Comparison**: Stability vs single-run evaluation  

### Key Advantages of Stability Selection

1. **Robustness**: Less sensitive to data perturbations
2. **Reproducibility**: Consistent feature sets across runs
3. **Generalization**: Better performance on new data
4. **Control**: Adjustable threshold for strictness
5. **Interpretability**: Stability scores show confidence

### When to Use Stability Selection

✅ Small sample sizes (n < 200)  
✅ High-dimensional data (p >> n)  
✅ Need for reproducible biomarkers  
✅ Clinical/translational research  
✅ Regulatory requirements (FDA)  

### 💡 Google Drive Integration Benefits

- **Checkpointing**: Never lose progress if Colab disconnects
- **Persistence**: All results saved automatically
- **Collaboration**: Share Drive folder with team
- **Reproducibility**: Complete analysis trail saved

### 📚 Next Steps

- **[06_model_interpretation.ipynb](06_model_interpretation.ipynb)** - Interpret your models
- **[07_complete_workflow.ipynb](07_complete_workflow.ipynb)** - Full analysis pipeline

---

**Stable features = reproducible science! 🧬🔬**